In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import myslack
import datetime as dt
import pickle
import time
import re
from tqdm import trange

In [2]:
driver = webdriver.Chrome()  
url = 'http://produce48.mnet.com/pc/profile/0'
driver.get(url) 

In [98]:
review_count = int(driver.find_element_by_xpath('//*[@id="comment"]/div/h2/strong').text)
comment_page_count = int(review_count/20)+1
review_count, comment_page_count

(889, 45)

In [99]:
def open_all_comment():
    print('이전 댓글 모두 불러오기 시작...')    
    for _ in trange(comment_page_count):
        try:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/a').click()
            time.sleep(0.3)
        except Exception as e:
            print(e)
            time.sleep(1)
            pass
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    print('이전 댓글 모두 불러오기 성공!')

In [100]:
def make_produce48_df(trainee_name):
    print(trainee_name+'의 댓글 수: '+str(review_count))
    open_all_comment()
    
    print('making dataframe...')
    df = pd.DataFrame(columns=['User_ID','Comment','Date'])
    for i in trange(review_count):
        try:
            df.loc[len(df)] = { 
            'User_ID':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/strong'.format(i+1)).text,
            'Comment':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/p'.format(i+1)).text,
            'Date':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/span'.format(i+1)).text.split(' ')[0],
            }
        except Exception as e:
            print(e)
            pass
    print('making dataframe done!')
    return df

In [101]:
def make_df_name():
    eng_name = \
    re.sub("[^a-zA-Z]"," ",driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]').text).strip()
    result = eng_name.replace(' ','_')
    return result

name = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]/span').text
agency = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/h4/span').text

make_df_name(), name, agency

('HASEGAWA_RENA', '하세가와 레나', 'NGT48')

In [102]:
%time HASEGAWA_RENA = make_produce48_df(name)

  0%|          | 0/45 [00:00<?, ?it/s]

하세가와 레나의 댓글 수: 889
이전 댓글 모두 불러오기 시작...


  9%|▉         | 4/45 [00:21<03:37,  5.31s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="comment"]/div/div[2]/div/a"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)



 11%|█         | 5/45 [00:22<02:59,  4.48s/it]

...

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="comment"]/div/div[2]/div/a"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)



  0%|          | 1/889 [00:00<02:31,  5.87it/s]

이전 댓글 모두 불러오기 성공!
making dataframe...


100%|██████████| 889/889 [02:41<00:00,  5.50it/s]

making dataframe done!
CPU times: user 6.85 s, sys: 619 ms, total: 7.47 s
Wall time: 3min 23s


In [103]:
HASEGAWA_RENA.shape

(889, 3)

In [109]:
HASEGAWA_RENA.tail()

,User_ID,Comment,Date
884,시조자,점만 빼면 상당히 괜찮을거 같은데..,2018-05-20
885,Kyoungsurk Kam,자세 좋네요^^..좋은 결과 있기를^^,2018-05-17
886,장일섭,자세가 제대로네 와 !!,2018-05-16
887,Takahashi Kaen,https://m.youtube.com/watch?v=sksx8XgKX5Q NGT4...,2018-05-16
888,TAKu,https://youtu.be/acIEE3_Nr64 오타니도 놀란 하세가와 레나의 ...,2018-05-16


In [110]:
pickle.dump(HASEGAWA_RENA, open('./produce48/NGT48/HASEGAWA_RENA.pkl','wb'))

In [111]:
HASEGAWA_RENA_df = pickle.load(open('./produce48/NGT48/HASEGAWA_RENA.pkl','rb'))

In [112]:
myslack.send_slack('{} finish!'.format(make_df_name()))

<Response [200]>


In [113]:
HASEGAWA_RENA_df.tail()

,User_ID,Comment,Date
884,시조자,점만 빼면 상당히 괜찮을거 같은데..,2018-05-20
885,Kyoungsurk Kam,자세 좋네요^^..좋은 결과 있기를^^,2018-05-17
886,장일섭,자세가 제대로네 와 !!,2018-05-16
887,Takahashi Kaen,https://m.youtube.com/watch?v=sksx8XgKX5Q NGT4...,2018-05-16
888,TAKu,https://youtu.be/acIEE3_Nr64 오타니도 놀란 하세가와 레나의 ...,2018-05-16


In [663]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기